In [1]:
from recom.datasets import load_ml_small_rating

# load data
# not that I use leave-one-out method to construct the testing set, where
# the latest rated item is masked and added to the testing set as an evaluation.
dataset = load_ml_small_rating(need_raw=True, time_ord=True, test_perc=0.1)

# load features
ratings = dataset['raw']
ratings_train_dict = dataset['train_dict']
ratings_test_dict = dataset['test_dict']
n_user = dataset['n_user']
n_item = dataset['n_item']
user2ix = dataset['user2ix']
ix2user = dataset['ix2user']
item2ix = dataset['item2ix']
ix2item = dataset['ix2item']

del dataset

print(f'Users: {n_user}, Items: {n_item}. Sparsity: {round(1-len(ratings)/n_user/n_item, 4)}')
print(f'User reduced from {len(user2ix.keys())} to {len(ratings_train_dict.keys())}')

Users: 609, Items: 9562. Sparsity: 0.983
User reduced from 609 to 608


In [2]:
import torch.nn as nn
from torch import Tensor, LongTensor
from torch.nn.functional import logsigmoid


def BPRLoss(gap):
    return -logsigmoid(gap)
    

class BPR(nn.Module):
    def __init__(self, n_user, n_item
                 , k_dim, std_user, std_item):
        from torch import sigmoid

        super(BPR, self).__init__()
        # embeddings
        self.embedding_user = nn.Embedding(n_user, k_dim)
        self.embedding_item = nn.Embedding(n_item, k_dim)
        # init param
        nn.init.normal_(self.embedding_user.weight, mean=0, std=std_user)
        nn.init.normal_(self.embedding_item.weight, mean=0, std=std_item)

    def forward(self, user, pos_item, neg_item):
        pos_score = (self.embedding_user(user) 
                     * self.embedding_item(pos_item)).sum(1)
        neg_score = (self.embedding_user(user) 
                     * self.embedding_item(neg_item)).sum(1)
        return pos_score-neg_score

    def pred_all(self, ):
        return self.embedding_user.weight \
               @ self.embedding_item.weight.T

In [3]:
def naive_pairwise_loader(rat_dict, items, batch_size, neg_size=None
                    , random_sampling=True, user_size=None, pos_size=None
                    , user_neg_dict=None):
    
    from random import choices
    from torch.utils.data import DataLoader


    if not isinstance(items, set):
        all_items = set(items)
    all_items = items
    
    train_data = []

    if not random_sampling: # goover all dataset
        for user in rat_dict:
            pos_items = list(rat_dict[user].keys())
            neg_candidates = list(all_items - set(pos_items)) if user_neg_dict is None \
                             else user_neg_dict[user]
            neg_items = choices(neg_candidates, k=len(pos_items)*neg_size)
            u = [user]*len(pos_items)*neg_size
            pos_items *= neg_size
            train_data.extend(zip(u, pos_items, neg_items))           
                
    else:
        users = choices(list(rat_dict.keys()), k=user_size)
        for user in users:
            neg_candidates = list(all_items - set(rat_dict[user])) if user_neg_dict is None \
                             else user_neg_dict[user]
            pos_items = choices(list(rat_dict[user].keys()), k=pos_size)
            neg_items = choices(neg_candidates, k=pos_size*neg_size)

            # duplicate
            u = [user] * pos_size*neg_size
            pos_items *= neg_size
            train_data.extend(zip(u, pos_items, neg_items))
        
    return DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)


# things for sampling
items = list(range(n_item))
# first define the dictionary can accelerate sampling efficiency
user_neg_dict = {
    u:list(set(items)-set(ratings_train_dict[u].keys()))
     for u in ratings_train_dict
}

# DL1: roll over all data
dl_roll = naive_pairwise_loader(
    rat_dict=ratings_train_dict, items=items, user_neg_dict=user_neg_dict
    , random_sampling=False, neg_size=4
    , batch_size=128
)
# DL2: sampling by parameters
dl_sample = naive_pairwise_loader(
    rat_dict=ratings_train_dict, items=items, user_neg_dict=user_neg_dict
    , random_sampling=True, neg_size=4
    , user_size=256, pos_size=64
    , batch_size=128
)

In [8]:
import time
from torch import autograd, LongTensor, device
from torch import optim
import numpy as np


def generate_testing_candidates(rating_train, n_item, n=None):
    from random import choices
    
    items = list(range(n_item))

    testing_cand = {
        u: choices(list(set(items)-set(rating_train[u].keys())), k=n) if n is not None
           else list(set(items)-set(rating_train[u].keys()))
        for u in rating_train
    }
    return testing_cand

from torch import argsort

results = {u:[] for u in ratings_train_dict}


def train_model(model, opt, rat_train, n_items
                , use_random_sampling=True, neg_size=4
                , user_size=256, pos_size=32
                , use_cuda=False, n_epochs=64, batch_size=256
                , test_dict=None, metrics=None, k=None
                , report_interval=1):

    if use_cuda:
        compute_device = device('cuda')
        model.cuda()
    else:
        compute_device = device('cpu')

    # things for sampling
    items = list(range(n_item))
    # first define the dictionary can accelerate sampling efficiency
    user_neg_dict = {
        u:list(set(items)-set(rat_train[u].keys()))
        for u in rat_train}

    train_loss_by_ep = []
    test_rmse_by_ep = []

    # place holder for metric
    if metrics is not None:
        metrics_at_k = {metric[0]:[] for metric in metrics.items()} 
        test_cands = generate_testing_candidates(ratings_train_dict, n_item, n=100)

    t0 = time.time()
    for epoch in range(n_epochs):
        train_data = dl_sample = naive_pairwise_loader(
                        rat_dict=rat_train, items=items
                        , user_neg_dict=user_neg_dict
                        , random_sampling=use_random_sampling
                        , neg_size=neg_size
                        , user_size=user_size, pos_size=pos_size
                        , batch_size=batch_size
                    )

        ep_loss = []
        for i, batch in enumerate(train_data):
            user, pos_item, neg_item = batch

            model.zero_grad()

            user = autograd.Variable(LongTensor(user)).to(compute_device)
            pos_item = autograd.Variable(LongTensor(pos_item)).to(compute_device)
            neg_item = autograd.Variable(LongTensor(neg_item)).to(compute_device)

            preds = model(user, pos_item, neg_item)
            loss = BPRLoss(gap=preds) # todo

            loss.sum().backward()
            opt.step()
            ep_loss.extend(loss.data.to(compute_device).tolist())
            
        train_loss_by_ep.append(np.mean(ep_loss))

        # compute testing result
        preds = bpr.pred_all().to('cpu')
        for u in results:
            pred_items = Tensor([preds[u][i] for i in test_cands[u]])
            results[u] = [test_cands[u][ix] for ix in argsort(-pred_items)[:100]]

        for metric in metrics_at_k:
            metrics_at_k[metric].append(metrics[metric](k, ratings_test_dict, results))

        if report_interval > 0 \
                and ((epoch+1) % report_interval == 0):
            
            t1=time.time()
            print(f'Epoch: {epoch+1}, Time: {round(t1-t0,2)},  /Average train loss {round(sum(train_loss_by_ep[-report_interval:])/report_interval, 5)}')
            average_metrics = {metric:round(sum(metrics_at_k[metric][-report_interval:])/report_interval, 5) for metric in metrics_at_k}
            test_metrics = ' '.join(f'{m_items[0]}:{m_items[1]}' for m_items in average_metrics.items())
            print(f'\t\t\t/Average test metric at {k}: {test_metrics}')
            t0=time.time()

    # finish traniing, send to cpu anyway
    model = model.to('cpu') 

    if test_dict is not None:
        return model, train_loss_by_ep, metrics_at_k

    return model, train_loss_by_ep


# from recom.model.pairwise import BPR
from torch import optim
from recom.eval.metrics import map, hit_rate, ndcg
import warnings
warnings.filterwarnings("ignore")

K_DIM=64
STD_USER=1
STD_ITEM=1
NEG_SIZE=16 # 4
USER_SIZE=512
POS_SIZE=64
USE_CUDA=True
N_EPOCHES=12
BATCH_SIZE=512
INTERVAL=1

bpr = BPR(
    n_user=n_user, n_item=n_item
    , k_dim=K_DIM
    , std_user=STD_USER
    , std_item=STD_ITEM
)
# leave one out
# opt = optim.Adam(bpr.parameters(), lr=0.002, weight_decay=0.1) # :mAP:0.0008 hit_rate:0.07566 ndcg:0.00919
# leave last 10% chronologically
# optim.Adam(bpr.parameters(), lr=0.001, weight_decay=0.05) # mAP:0.00445 hit_rate:0.12336 ndcg:0.0537
opt = optim.Adam(bpr.parameters(), lr=0.001, weight_decay=0.05)
bpr, train_loss_by_ep, test_rmse_by_ep = train_model(
    model=bpr, opt=opt, rat_train=ratings_train_dict
    , n_items=n_item, use_random_sampling=True
    , neg_size=NEG_SIZE
    , user_size=USER_SIZE, pos_size=POS_SIZE
    , use_cuda=USE_CUDA, n_epochs=N_EPOCHES, batch_size=BATCH_SIZE
    , test_dict=ratings_test_dict, metrics={'mAP': map, 'hit_rate':hit_rate, 'ndcg':ndcg}, k=10
    , report_interval=INTERVAL
)

Epoch: 1, Time: 6.13,  /Average train loss 2.90092
			/Average test metric at 10: mAP:0.00079 hit_rate:0.05099 ndcg:0.00956
Epoch: 2, Time: 5.01,  /Average train loss 1.06781
			/Average test metric at 10: mAP:0.00124 hit_rate:0.06908 ndcg:0.01455
Epoch: 3, Time: 5.65,  /Average train loss 0.5536
			/Average test metric at 10: mAP:0.00192 hit_rate:0.08882 ndcg:0.02236
Epoch: 4, Time: 4.87,  /Average train loss 0.32191
			/Average test metric at 10: mAP:0.00234 hit_rate:0.10362 ndcg:0.02848
Epoch: 5, Time: 5.37,  /Average train loss 0.22811
			/Average test metric at 10: mAP:0.0029 hit_rate:0.10033 ndcg:0.03556
Epoch: 6, Time: 4.89,  /Average train loss 0.19619
			/Average test metric at 10: mAP:0.00374 hit_rate:0.125 ndcg:0.04643
Epoch: 7, Time: 4.78,  /Average train loss 0.19196
			/Average test metric at 10: mAP:0.0043 hit_rate:0.12829 ndcg:0.051
Epoch: 8, Time: 4.75,  /Average train loss 0.1909
			/Average test metric at 10: mAP:0.00457 hit_rate:0.12171 ndcg:0.05523
Epoch: 9, Time: 